In [ ]:
import pandas as pd


!wget http://ptrckprry.com/course/ssd/data/positive-words.txt #Opinion Lexicon: Positive Words; source: http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
!wget http://ptrckprry.com/course/ssd/data/negative-words.txt #Opinion Lexicon: Negative Words; soruce: http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html


filePath1 = "/content/positive-words.txt"
filePath2 = "/content/negative-words.txt"


posWords_df = pd.read_csv(filePath1,sep='\t',header=None, names=['words'],encoding='latin-1',skiprows=[i for i in range(0,34)]) #Latin-1 (ISO-8859-1) character set. Use chardet lib if needed
negWords_df = pd.read_csv(filePath2,sep='\t',header=None, names=['words'],encoding='latin-1',skiprows=[i for i in range(0,34)])


print(f'Number of posWords: {len(posWords_df)} Number of negWords: {negWords_df.size}')
display(posWords_df.head(10))
display(negWords_df.head(10))


--2025-11-07 09:07:38--  http://ptrckprry.com/course/ssd/data/positive-words.txt
Resolving ptrckprry.com (ptrckprry.com)... 185.199.108.153, 185.199.109.153, 185.199.111.153, ...
Connecting to ptrckprry.com (ptrckprry.com)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ptrckprry.com/course/ssd/data/positive-words.txt [following]
--2025-11-07 09:07:38--  https://ptrckprry.com/course/ssd/data/positive-words.txt
Connecting to ptrckprry.com (ptrckprry.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20630 (20K) [text/plain]
Saving to: ‘positive-words.txt’

positive-words.txt  100%[===================>]  20.15K  --.-KB/s    in 0s      

2025-11-07 09:07:38 (114 MB/s) - ‘positive-words.txt’ saved [20630/20630]

--2025-11-07 09:07:38--  http://ptrckprry.com/course/ssd/data/negative-words.txt
Resolving ptrckprry.com (ptrckprry.com)... 185.199.108.153, 185.199.109.153, 185.199.111.15

,words
0,a+
1,abound
2,abounds
3,abundance
4,abundant
5,accessable
6,accessible
7,acclaim
8,acclaimed
9,acclamation


,words
0,2-faced
1,2-faces
2,abnormal
3,abolish
4,abominable
5,abominably
6,abominate
7,abomination
8,abort
9,aborted


Task 1

In [ ]:
def sentimentAnalyser(k):
  print(f'string = {k}')
  k=k.lower()
  w1=k.split()
  con=0
  tot=len(w1)
  p1,n1=0,0
  for i in w1:
    if i in posWords_df.values:
      p1=p1+1
    elif i in negWords_df.values:
      n1=n1+1
  if p1>n1:
    print('positive sentiment')
    con=p1/tot
    print(f'confidence: {1-con}')
  elif p1<n1:
    print('negative sentiment')
    con=n1/tot
    print(f'confidence: {1-con}')
  elif p1==n1:
    print('neutral sentiment')
    con=p1/tot
    print(f'confidence: {1-con}')
  print('------------------------------------')

sentimentAnalyser("NLP is cool")
sentimentAnalyser("NLP is cool and useful")
sentimentAnalyser("NLP is hard")
sentimentAnalyser("NLP is hard and useless")
sentimentAnalyser("NLP stands for Natural language processing")

string = NLP is cool
positive sentiment
confidence: 0.6666666666666667
------------------------------------
string = NLP is cool and useful
positive sentiment
confidence: 0.6
------------------------------------
string = NLP is hard
negative sentiment
confidence: 0.6666666666666667
------------------------------------
string = NLP is hard and useless
negative sentiment
confidence: 0.6
------------------------------------
string = NLP stands for Natural language processing
neutral sentiment
confidence: 1.0
------------------------------------


Task 2

In [ ]:
def naiveBayesClassifier(trainingSet, testSet):

    megadocGB, megadocIE = "", ""
    tgb, tie = 0, 0
    for text, label in trainingSet:
        if label == 'GB':
            tgb += 1
            megadocGB += " " + text
        elif label == 'IE':
            tie += 1
            megadocIE += " " + text

    print(f'megaDocGB = {megadocGB}')
    print(f'megaDocIE = {megadocIE}')
    megadocGB = megadocGB.lower()
    megadocIE = megadocIE.lower()
    print(f'ProbGB = {tgb / (tgb + tie):.2f}  ProbIE = {tie / (tgb + tie):.2f}')
    gbword = megadocGB.split()
    ieword = megadocIE.split()
    tgb = len(gbword)
    tie = len(ieword)
    uniquegb = set(gbword)
    uniqueie = set(ieword)
    gbbow = {word: gbword.count(word) for word in uniquegb}
    iebow = {word: ieword.count(word) for word in uniqueie}
    vocab = list(set(gbword + ieword))
    vsize = len(vocab)
    v1 = {word: (gbbow.get(word, 0) + iebow.get(word, 0)) for word in vocab}
    print("GB_BOW =", gbbow)
    print("IE_BOW =", iebow)
    print(f'V = {v1}')
    print(f'|V| = {vsize}')
    print("---------------------------------------------------------------------------")


    for text, _ in testSet:
        text = text.lower()
        words = text.split()
        pGB = tgb / (tgb + tie)
        pIE = tie / (tgb + tie)

        print(f'\nTest Sentence: "{text}"')
        for word in words:
            if word in v1:
                pwGB = (gbbow.get(word, 0) + 1) / (tgb + vsize)
                pwIE = (iebow.get(word, 0) + 1) / (tie + vsize)
                pGB *= pwGB
                pIE *= pwIE
                print(f'  word = "{word}" , P(w|GB) = {pwGB:.6f} , P(w|IE) = {pwIE:.6f}')
            else:
                print(f'  word = "{word}" , ignored (out of vocabulary)')

        print(f'\n DocProbgb = {pGB:.6f} DocProbIE = {pIE:.6f}')
        print("  Inferred class = ", "GB" if pGB > pIE else "IE")
        print("---------------------------------------------------------------------------")




In [ ]:

trainingSet = [('London is the Capital of GB','GB'),('Oxford is a city in GB','GB'),('Dublin is the capital of Ireland','IE'),('Limerick is a city in Ireland','IE')]
testSet = [('University of Limerick','?'),('University College Dublin','?'),('Imperial College London','?'),('University of Oxford','?'),('Ireland & GB','?')]
naiveBayesClassifier(trainingSet,testSet)



megaDocGB =  London is the Capital of GB Oxford is a city in GB
megaDocIE =  Dublin is the capital of Ireland Limerick is a city in Ireland
ProbGB = 0.50  ProbIE = 0.50
GB_BOW = {'capital': 1, 'the': 1, 'in': 1, 'city': 1, 'london': 1, 'is': 2, 'of': 1, 'gb': 2, 'oxford': 1, 'a': 1}
IE_BOW = {'capital': 1, 'the': 1, 'ireland': 2, 'in': 1, 'dublin': 1, 'city': 1, 'is': 2, 'limerick': 1, 'of': 1, 'a': 1}
V = {'capital': 2, 'the': 2, 'in': 2, 'ireland': 2, 'dublin': 1, 'city': 2, 'london': 1, 'is': 4, 'of': 2, 'gb': 2, 'limerick': 1, 'oxford': 1, 'a': 2}
|V| = 13
---------------------------------------------------------------------------

Test Sentence: "university of limerick"
  word = "university" , ignored (out of vocabulary)
  word = "of" , P(w|GB) = 0.080000 , P(w|IE) = 0.080000
  word = "limerick" , P(w|GB) = 0.040000 , P(w|IE) = 0.080000

 DocProbgb = 0.001600 DocProbIE = 0.003200
  Inferred class =  IE
---------------------------------------------------------------------------

Te